<a href="https://colab.research.google.com/github/MyrtooS/Explainability-Methods-for-Brain-Tumor-Classification/blob/main/load_image_train_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive'
folder='/content/drive/My Drive/MRI Classification/mri_kaggle'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive


In [ ]:
import cv2
import os
import glob
from PIL import Image
import os, os.path
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.datasets 
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
# from tensorflow.keras import applications
# from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
list_of_files_train = []
path_train = "/content/drive/My Drive/MRI Classification/mri_kaggle/Training/"
for file in os.listdir(path_train): #reading all sub file directories
    mri_path_train= path_train+str(file)+'/*.jpg'
    list_of_files_train.append(mri_path_train)

print(list_of_files_train)


['/content/drive/My Drive/MRI Classification/mri_kaggle/Training/pituitary_tumor/*.jpg', '/content/drive/My Drive/MRI Classification/mri_kaggle/Training/glioma_tumor/*.jpg', '/content/drive/My Drive/MRI Classification/mri_kaggle/Training/meningioma_tumor/*.jpg', '/content/drive/My Drive/MRI Classification/mri_kaggle/Training/no_tumor/*.jpg']


In [ ]:
array_images_train = []
array_labels_train = []
    
label_counter_train=0
for i in list_of_files_train:
  images_train = glob.glob(i)
        
  for image in images_train:
    with open(image, 'rb') as r_image:
           
            
      img = Image.open(image)
      resized_image = img.resize((224,224))
      resized_image=np.array(resized_image)
                
      array_images_train.append(resized_image)
      array_labels_train.append(label_counter_train)  
         
  label_counter_train = label_counter_train + 1 

In [ ]:
x_train = np.array(array_images_train)
y_train = np.array(array_labels_train)
y_train = tensorflow.keras.utils.to_categorical(y_train, 4)

y1_train = y_train.reshape(4054,-1)
x1_train = x_train.reshape(4054,-1)

sm = SMOTE(random_state=42)
X_smote, y_smote = sm.fit_resample(x1_train, y1_train)
x_train  = X_smote.reshape(4524,224,224,3)


X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_smote, test_size=0.2, random_state=1, stratify = y_smote)

1η Προσέγγιση - Κατηγοριοποίηση με 4 κλάσεις

1ο custom μοντέλο

In [ ]:
model = tensorflow.keras.Sequential() 
model.add(layers.Input(shape=x_train[0].shape))
model.add(layers.Conv2D(3, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.AveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
X_train_gap = X_train.astype('float32') / 255.0
X_test_gap = X_test.astype('float32') / 255.0

In [ ]:
sgd = tensorflow.keras.optimizers.SGD(learning_rate= 0.001)
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_gap, Y_train, batch_size=32, epochs=13)

In [ ]:
model.save('/content/drive/My Drive/MRI Classification/gap_4.h5')

In [ ]:
gap_model = tf.keras.models.load_model('/content/drive/My Drive/MRI Classification/gap_4.h5')

In [ ]:
rounded_predictions = np.argmax(gap_model.predict(X_test_gap), axis=-1)
rounded_predictions[1]
rounded_labels=np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_labels, rounded_predictions)
print(cm)

In [ ]:
accuracy = accuracy_score(rounded_predictions, rounded_labels)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions, rounded_labels, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions, rounded_labels,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions, rounded_labels,average = None)
print('F1 score: ',f1)

2o custom μοντέλο

In [ ]:
model = tensorflow.keras.Sequential() 
model.add(layers.Input(shape=x_train[0].shape))
model.add(layers.Conv2D(3, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))

model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))


model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))


model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(4096 , activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
X_train_nogap = X_train.astype('float32') / 255.0
X_test_nogap = X_test.astype('float32') / 255.0

In [ ]:
sgd = tensorflow.keras.optimizers.SGD(learning_rate= 0.001)
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_nogap, Y_train_nogap, batch_size=32, epochs=13)

In [ ]:
score = vgg_model.evaluate(X_test_vgg,Y_test)

In [ ]:
model.save('/content/drive/My Drive/MRI Classification/gap_4.h5')

In [ ]:
nogap_model = tf.keras.models.load_model('/content/drive/My Drive/MRI Classification/No_gap_multi.h5')

In [ ]:
rounded_predictions = np.argmax(nogap_model.predict(X_test_nogap), axis=-1)
rounded_predictions[1]
rounded_labels=np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_labels, rounded_predictions)
print(cm)

In [ ]:
accuracy = accuracy_score(rounded_predictions, rounded_labels)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions, rounded_labels, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions, rounded_labels,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions, rounded_labels,average = None)
print('F1 score: ',f1)

VGG16

In [ ]:
vgg = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))
vgg.trainable = False 
x = layers.Flatten()(vgg.output)
x1 = layers.Dense(units=4096,activation="relu")(x)
x2 = layers.Dense(units=4,name="dense_final")(x1)
x4 = layers.Activation("softmax")(x2)

vgg_model = tf.keras.Model(vgg.input, [x4])

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
X_train_vgg = tf.keras.applications.vgg16.preprocess_input(X_train, data_format=None)
X_test_vgg = tf.keras.applications.vgg16.preprocess_input(X_test, data_format=None)

In [ ]:
sgd = tensorflow.keras.optimizers.SGD(learning_rate= 0.001)
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_vgg, Y_train, batch_size=16, epochs=5)

In [ ]:
score = vgg_model.evaluate(X_test_vgg,Y_test)

In [ ]:
vgg_model.save('/content/drive/My Drive/MRI Classification/Multi_VGG16_1.h5')

In [ ]:
vgg_model = tf.keras.models.load_model('/content/drive/My Drive/MRI Classification/Multi_VGG16_1.h5')

In [ ]:
from sklearn.metrics import confusion_matrix
rounded_predictions = np.argmax(vgg_model.predict(X_test_vgg), axis=-1)
rounded_predictions[1]
rounded_labels=np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_labels, rounded_predictions)
print(cm)

[[222   1   1   2]
 [ 15 200   9   2]
 [ 33  34 148  12]
 [  7   4   1 214]]


In [ ]:
accuracy = accuracy_score(rounded_predictions, rounded_labels)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions, rounded_labels, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions, rounded_labels,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions, rounded_labels,average = None)
print('F1 score: ',f1)

Accuracy:  0.8662983425414365
Precision:  [0.98230088 0.88495575 0.65198238 0.94690265]
Recall:  [0.80144404 0.83682008 0.93081761 0.93043478]
F1 score:  [0.88270378 0.86021505 0.76683938 0.93859649]


EfficientNetB0


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
efficientnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape= (224,224,3))

x = layers.Flatten()(efficientnet.output)
x = layers.Dense(units=512,activation="relu")(x)
x = layers.Dense(units=1,name = "dense_final")(x)
x = layers.Activation("softmax")(x)

eff_model = tf.keras.Model(efficientnet.input, [x])

In [ ]:
eff_model.save('/content/drive/My Drive/MRI Classification/Multi_EfficientNet.h5')

In [ ]:
eff_model = tf.keras.models.load_model('/content/drive/My Drive/MRI Classification/Multi_EfficientNet.h5')

In [ ]:
X_train_eff = tf.keras.applications.vgg16.preprocess_input(X_train, data_format=None)
X_test_eff = tf.keras.applications.vgg16.preprocess_input(X_test, data_format=None)

In [ ]:
sgd = tensorflow.keras.optimizers.SGD(learning_rate= 0.001)
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_eff, Y_train, batch_size=16, epochs=5)

In [ ]:
score = eff_model.evaluate(X_test_eff,Y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
rounded_predictions = np.argmax(eff_model.predict(X_test_eff), axis=-1)
rounded_predictions[1]
rounded_labels=np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_labels, rounded_predictions)
print(cm)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(rounded_predictions, rounded_labels)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions, rounded_labels, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions, rounded_labels,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions, rounded_labels,average = None)
print('F1 score: ',f1)

2η Προσέγγιση - Δυαδική Κατηγοριοποίηση

In [ ]:
list_of_files_train_binary = []
path_train = "/content/drive/My Drive/MRI Classification/Brain Tumor Data Set/"
for file in os.listdir(path_train): #reading all sub file directories
    mri_path_train= path_train+str(file)+'/*.jpg'
    list_of_files_train_binary.append(mri_path_train)

print(list_of_files_train_binary)

In [ ]:
array_images_train_binary = []
array_labels_train_binary = []
    
label_counter_train_binary=0
for i in list_of_files_train_binary:
  images_train_binary = glob.glob(i)
        
  for image in images_train_binary:
    with open(image, 'rb') as r_image:
           
            
      img = Image.open(image)
      resized_image = img.resize((224,224))
      resized_image=np.array(resized_image)
      if len(resized_image.shape) == 3:
        if resized_image.shape[0] == 224 and resized_image.shape[1] == 224 and resized_image.shape[2] == 3:    
                    
          array_images_train_binary.append(np.array(resized_image))
          array_labels_train_binary.append(label_counter_train) 

      
  label_counter_train_binary = label_counter_train_binary + 1 


In [ ]:
x_train_binary = np.array(array_images_train)
y_train_binary = np.array(array_labels_train)

from sklearn.model_selection import train_test_split
X_train_binary, X_test_binary, Y_train_binary, Y_test_binary = train_test_split(x_train_binary, y_train_binary, test_size=0.2, random_state=1,stratify = y_train_binary)

1o custom μοντέλο

In [ ]:
model = tensorflow.keras.Sequential() 
model.add(layers.Input(shape=x_train[0].shape))
model.add(layers.Conv2D(3, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.AveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

In [ ]:
X_train_bgap = X_train_binary.astype('float32') / 255.0
X_test_bgap = X_test_binary.astype('float32') / 255.0

In [ ]:
sgd = tensorflow.keras.optimizers.SGD(learning_rate= 0.001)
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.00001)
model.compile(optimizer=adam ,loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_binary, Y_train_binary, batch_size=16, epochs=20)

In [ ]:
score = model.evaluate(X_test_bgap,Y_test)

In [ ]:
model.save('/content/drive/My Drive/MRI Classification/Binary_GAP.h5')

In [ ]:
bgap_model = tf.keras.models.load_model('/content/drive/My Drive/MRI Classification/Binary_GAP.h5')

In [ ]:
y_pred = bgap_model.predict(X_test_binary)
y_pred = (y_pred > 0.5)
  
cm = confusion_matrix(Y_test_binary, y_pred)

print(cm)

In [ ]:
accuracy = accuracy_score(rounded_predictions, rounded_labels)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions, rounded_labels, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions, rounded_labels,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions, rounded_labels,average = None)
print('F1 score: ',f1)

2o custom μοντέλο 

In [ ]:
X_train_b_nogap = X_train_binary.astype('float32') / 255.0

In [ ]:
X_test_b_nogap = X_test_binary.astype('float32') / 255.0

In [ ]:
model = tensorflow.keras.Sequential() 
model.add(layers.Input(shape=x_train[0].shape))
model.add(layers.Conv2D(3, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))

model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))


model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))


model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(4096 , activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
sgd = tensorflow.keras.optimizers.SGD(learning_rate= 0.001)
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.00001)
model.compile(optimizer=adam ,loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_b_nogap, Y_train, batch_size=32, epochs=8)

In [ ]:
model.save('/content/drive/My Drive/MRI Classification/Binary_Νο_GAP.h5')


In [ ]:
no_gap_model_binary = tf.keras.models.load_model('/content/drive/My Drive/MRI Classification/Binary_Νο_GAP.h5')


In [ ]:
score = no_gap_model_binary.evaluate(X_test_b_nogap,Y_test)

In [ ]:
y_pred = no_gap_model_binary.predict(X_test_b_nogap)
y_pred = (y_pred[:,0] > 0.5)
    
cm = confusion_matrix(Y_test, y_pred)

print(cm)

EfficientNetB0


In [ ]:

efficientnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape= (224,224,3))

x = layers.Flatten()(efficientnet.output)

x = layers.Dense(units=512,activation="relu")(x)

x = layers.Dense(units=1,name = "dense_final")(x)
x = layers.Activation("sigmoid")(x)

model = tf.keras.Model(efficientnet.input, [x])

In [ ]:
sgd = tensorflow.keras.optimizers.SGD(learning_rate= 0.001)
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam ,loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_binary, Y_train_binary, batch_size=16, epochs=5)

In [ ]:
model.save('/content/drive/My Drive/MRI Classification/Binary_EfficientNetB0.h5')

In [ ]:
b_eff_model = tf.keras.models.load_model('/content/drive/My Drive/MRI Classification/Binary_EfficientNetB0.h5')

In [ ]:
score = b_eff_model.evaluate(X_test_binary,Y_test)

In [ ]:
y_pred = b_eff_model.predict(X_test_binary)
y_pred = (y_pred > 0.5)
    
cm = confusion_matrix(Y_test_binary, y_pred)

print(cm)

Δεύτερο βοηθητικό μοντέλο

In [ ]:
y = b_eff_model.get_layer('dense_final').output
y = tf.math.scalar_mul(0.2, y)
y = layers.Activation("sigmoid")(y)

b_eff_y_model = Model(inputs=b_eff_model.input, outputs=y)